In [1]:
#librarys
from iminuit import Minuit
import numpy as np

In [2]:
#functions used by this script
%run /home/joao/Documentos/LC/integratedfunctions/interpfunc.ipynb

In [3]:
def data(name):
    # receive the name of the file in the folder
    
    #time, flux, flux error and bands
    x_dat = []
    z_dat = []
    z_dat_error = []
    bands = []

    #reading the file
    file = open(name)
    lines = file.readlines()
    file.close()

    for line in lines:
        if line.startswith('#'): continue
        co=line.rstrip().replace('INDEF','Nan').split()

        x_dat.append(co[0])
        z_dat.append(co[2])
        z_dat_error.append(co[3])
        bands.append(co[1])

    #converting to float
    x_dat = [float(i) for i in x_dat]
    z_dat = [float(i) for i in z_dat]
    z_dat_error = [float(i) for i in z_dat_error]
    
    #redshift value
    redshift = float(co[6])

    #as we have a lot of bands in the reading file we exclude the repeated ones
    nonrepeatedbands = list(set(bands))

    #for each band we call the integrated flux functions
    interpfunc = []
    for ii in range(0,len(nonrepeatedbands)):
        
        #store in a list integf_m0, integf_m1 and integf_m2
        interpfunc.append(einterpfunctions(nonrepeatedbands[ii]))
  
        
    nx_dat = []
    nz_dat = []
    nz_dat_error = []
        
    #readshift correction the data
    for i in range(0,len(x_dat)):

        nx_dat.append(x_dat[i]*(1+redshift))
        nz_dat.append(z_dat[i]*(1+redshift)**3)
        nz_dat_error.append(z_dat_error[i]*(1+redshift)**3)

    #chiquad def function
    def chiquadvalue(t0, x0, x1, x2):


        
        #initial value of chi quad
        chiquad = 0

        #print(nonrepeatedbands)
        #for each band
        for i in range(0,len(nonrepeatedbands)):
            
            #temporary time, flux and flux error lists for each band separately
            x_dat_temp = []
            z_dat_temp = []
            z_dat_error_temp = []

            #index of the bands
            bandindex = [j for j, e in enumerate(bands) if e == nonrepeatedbands[i]]

        
            #temporary time, flux and error flux 
            for j in range(0,len(bandindex)):
            
                x_dat_temp.append(nx_dat[bandindex[j]])
                z_dat_temp.append(nz_dat[bandindex[j]])
                z_dat_error_temp.append(nz_dat_error[bandindex[j]])             
                
            #for each element of the temporary list evaluate chiquad and sum
            for k in range(0,len(x_dat_temp)):

                #restriction of the grid
                if -10 < x_dat_temp[k] - t0 < 50:
                    
                    chiquad += ((x0*(interpfunc[i][0](x_dat_temp[k] - t0) + x1*interpfunc[i][1](x_dat_temp[k] -  t0) + x2*interpfunc[i][2](x_dat_temp[k] -t0)) - z_dat_temp[k])/z_dat_error_temp[k])**2

        #return chi quad value
        return chiquad

    
    #index of the highest flux element to use of initial guess for the minimizer
    max_index = list(nz_dat).index(max(nz_dat))    
    print(nx_dat[max_index])
    #minimizer Minuit with the initial guess
    m = Minuit(chiquadvalue, t0 = nx_dat[max_index], x0 = 1, x1 = 0.0000001, x2 = 0.0000001, error_t0 = 1, error_x0 = 2, error_x1 = 2, error_x2 = 2, errordef=1, limit_x0 = (0,1), limit_x1 = (-0.00001, 0.00001), limit_x2 = (-0.00001, 0.000001))
    #m = Minuit(chiquadvalue, t0 = nx_dat[max_index], x0 = 1, x1 = 0.1, x2 = 0.1)

    #print(z_dat)    

    # run optimiser
    m.migrad()

    #values of parameters
    print(m.values) 
    
    #print(z_dat)    
    
    #chi quad/dof
    return m.fval / (len(x_dat) - len(m.values)), m.values, nonrepeatedbands, interpfunc, bands, nx_dat, nz_dat, nz_dat_error